In [ ]:
import ssl
import mlflow
import torch

from torch import nn
from torch.utils.data import DataLoader
from torchinfo import summary
from torchmetrics import Accuracy
from torchvision import datasets
from torchvision.transforms import ToTensor

ssl._create_default_https_context = ssl._create_unverified_context
device = torch.device('cpu')


# Modèle de classification d'image de mode avec PyTorch

In [ ]:
mlflow.end_run()

In [ ]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=64)

# Define the model.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


def train(dataloader, model, loss_fn, metrics_fn, optimizer):
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)
        accuracy = metrics_fn(pred, y)

        # Backpropagation.
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch
            mlflow.log_metric("loss", f"{loss:3f}", step=(batch // 100))
            mlflow.log_metric("accuracy", f"{accuracy:3f}", step=(batch // 100))
            print(
                f"loss: {loss:3f} accuracy: {accuracy:3f} [{current} / {len(dataloader)}]"
            )

epochs = 3
loss_fn = nn.CrossEntropyLoss()
metric_fn = Accuracy(task="multiclass", num_classes=10).to(device)
model = NeuralNetwork().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


with mlflow.start_run():
    params = {
        "epochs": epochs,
        "learning_rate": 1e-3,
        "batch_size": 64,
        "loss_function": loss_fn.__class__.__name__,
        "metric_function": metric_fn.__class__.__name__,
        "optimizer": "SGD",
    }
    # Log training parameters.
    mlflow.log_params(params)

    # Log model summary.
    with open("../model_summary.txt", "w") as f:
        f.write(str(summary(model)))
    mlflow.log_artifact("../model_summary.txt")

    for t in range(epochs):
        print(f"Epoch {t + 1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, metric_fn, optimizer)

    # Save the trained model to MLflow.


### Définition sure serveur MLFLOW

In [ ]:
mlflow.set_tracking_uri(f"http://model-platform.com/registry/test/")


## Enregistrement du modèle avec sa modèle card

In [ ]:
mlflow.pytorch.log_model(model, "custom_model", registered_model_name="torch_fashion_model")
mlflow.set_tag("mlflow.note.content", open("./model_card.md", "r").read())


In [ ]:

import mlflow.pyfunc

model_path = "../tmp/1742200769_test_torch_fashion_model_1/custom_model"
model = mlflow.pyfunc.load_model(model_path)
keras_pyfunc = mlflow.pyfunc.load_model(model_path)


In [ ]:
dataiter = iter(train_dataloader)
images, labels = next(dataiter)
images[0]
labels[0]

In [ ]:
import numpy as np
prediction = keras_pyfunc.predict(data = np.array(images[0]))
prediction

In [ ]:
images[0]